In [9]:
# data from https://jazzomat.hfm-weimar.de/download/downloads/wjazzd.db
import sqlite3
import pandas as pd

In [10]:
conn = sqlite3.connect("data/wjazzd.db")

query = """
    SELECT 
        melody.melid AS track_id, 
        melody.eventid - minmel.minid AS serial, 
        melody.pitch AS pitch, 
        melody.beat AS beat, 
        melody.bar AS bar, 
        sections.value AS chord 
    FROM melody 
    JOIN (
        SELECT 
            MIN(eventid) AS minid, 
            melid 
        FROM melody
        GROUP BY melid
    )
    AS minmel 
    ON melody.melid = minmel.melid 
    JOIN sections 
    ON sections.melid = melody.melid 
    WHERE 
        sections.type = 'CHORD' AND 
        sections.start <= melody.eventid - minmel.minid AND 
        sections.end >= melody.eventid - minmel.minid
;"""

df =  pd.read_sql_query(query, conn)

In [11]:
df.head()

,track_id,serial,pitch,beat,bar,chord
0,1,0,65.0,1,0,Bb6
1,1,1,63.0,2,0,Bb6
2,1,2,58.0,2,0,Bb6
3,1,3,61.0,3,0,Bb6
4,1,4,63.0,4,0,Bb6


In [12]:
# standardize chord notation

df["chord"] = df["chord"].replace('-', 'min', regex=True)
df["chord"] = df["chord"].replace('j', 'maj', regex=True)
df["chord"] = df["chord"].replace('\+', 'aug', regex=True)
df["chord"] = df["chord"].replace('o', 'dim', regex=True)

In [13]:
df.to_csv("melody_chords.csv")